In [ ]:
import functools
import itertools
import math
import re
import statistics
from functools import reduce
from pprint import pprint
from typing import (
    List,
    Optional,
    Generator,
    NamedTuple,
    Tuple,
    Set,
    Any,
    Dict,
    Union,
    Callable,
    Iterable,
    Sequence,
)

import rich.panel
import sklearn.base
from alive_progress import alive_bar
from numpy.distutils.misc_util import is_sequence
from tqdm import tqdm, trange
from log_symbols import LogSymbols

import overrides as overrides
import pandas as pds
import tabulate as tabulate
from collections import namedtuple
from sklearn import linear_model, metrics, preprocessing, model_selection, svm
from tabulate import tabulate
from pathlib import Path
import pickle
from halo import Halo
import random
import argparse
import sys
from sklearn.experimental import enable_halving_search_cv
from sklearn import neural_network, tree
from sklearn.model_selection import (
    HalvingGridSearchCV,
    RandomizedSearchCV,
    HalvingRandomSearchCV,
)
import scipy.stats as scistats

import rich.progress
import commons
from data_types import PickleOut
from rich.traceback import install as niceTracebacks
from rich.table import Table as rTable

from db_actions import db_actions
import numpy as np

from linRegPred import getSplits, getTransformers
from scipy.stats import rankdata
from sklearn.metrics import r2_score
import textdistance as TD
from multiprocessing import Pool
from sklearn.cluster import AffinityPropagation
from sklearn.cluster import KMeans

In [ ]:
with db_actions.connect() as conn:
    dF = pds.read_sql('SELECT * FROM "runtimeScorePredBase1000"', conn)
# print(dF)
all_x_cols = [
    "build-linux-kernel1",
    "fio2",
    "fio3",
    "fio4",
    "fio5",
    "fio6",
    "fio7",
    "fio8",
    "fio9",
    "iperf10",
    "iperf11",
    "iperf12",
    "iperf13",
    "john-the-ripper14",
    "john-the-ripper15",
    "ramspeed16",
    "ramspeed17",
    "ramspeed18",
    "ramspeed19",
    "ramspeed20",
    "ramspeed21",
    "ramspeed22",
    "ramspeed23",
    "ramspeed24",
    "ramspeed25",
    "stream26",
    "stream27",
    "stream28",
    "stream29",
    "pCpu",
    "cpus",
    "rss",
    "vmem",
    "rchar",
    "wchar",
    "syscr",
    "syscw",
]
filtered_x_cols = ['john-the-ripper15', 'ramspeed17', 'ramspeed18', 'ramspeed19', 'ramspeed20', 'ramspeed21',
                   'ramspeed23', 'ramspeed24', 'ramspeed25', 'pCpu', 'cpus', 'vmem', 'rchar', 'wchar', 'syscr',
                   'syscw']
x_cols = all_x_cols
y_cols = "realtime"

X: pds.DataFrame = dF[x_cols]
y: pds.DataFrame = dF[y_cols]

In [ ]:
from my_yaml import yaml_load

with open("nodeConfigIdLookup.yaml", "r") as f:
    t = yaml_load(f)
    nodeIDLUT = {v: k for k, v in t.items()}
dF["nodeName"] = dF["nodeConfig"].transform(lambda x: nodeIDLUT[x])

In [ ]:
awsGroupLUT = {
    "general": ["M7g", "Mac", "M6g", "M6i", "M6in", "M6a", "M5", "M5n", "M5zn", "M5a", "M4", "A1", "T4g", "T3",
                "T3a",
                "T2"],
    "compute": ["C7g", "C7gn", "C6i", "C6in", "C6a", "C6g", "C6gn", "C5", "C5n", "C5a", "C4"],
    "memory": ["R7g", "R7iz", "R6g", "R6i", "R6in", "R6a", "R5", "R5n", "R5b", "R5a", "R4", "X2gd", "X2idn",
               "X2iedn",
               "X2iezn", "X1", "X1e", "z1d"],
    "accelerated": ["P4", "P3", "P2", "DL1", "Trn1", "Inf2", "Inf1", "G5", "G5g", "G4dn", "G4ad", "G3", "F1",
                    "VT1"],
    "storage": ["Im4gn", "Is4gen", "I4i", "I3", "I3en", "D2", "D3", "D3en", "H1"],
    "HPC": ["Hpc6id", "Hpc6a"]
}

In [ ]:
awsInstanceGroupLUT = {i.lower(): k for k, v in awsGroupLUT.items() for i in v}

In [ ]:
dF["nodeGroup"] = dF["nodeName"].transform(lambda x: awsInstanceGroupLUT[x.split(".")[0]])

In [ ]:
usedAwsGroupLUT = {
    k: list(pds.unique(dF.query("nodeGroup==@k").nodeName.values)) for k in awsGroupLUT.keys()
}

In [ ]:
nodesPerGroup = {
    k: len(v) for k, v in usedAwsGroupLUT.items()
}

In [ ]:
import seaborn as sns

if False:
    #sns.boxplot(data=dF,y="realtime",x="taskName",hue="nodeGroup")
    for wf in dF.groupby(by="wfName"):
        wfname, data = wf
        print(wfname)
        grid = sns.FacetGrid(data, row="wfName", col="nodeGroup", sharex=False, sharey=True, margin_titles=True,
                             height=5)
        grid.map_dataframe(sns.boxplot, y="realtime", x="taskName")

# Find prototypes for each group

In [ ]:
true_ranks_df = pds.DataFrame()
for l, d in dF.groupby(by=["wfName", "taskName", "nodeName"]):
    wn, tn, nn = l
    true_ranks_df = true_ranks_df.append(
        {"wfName": wn, "taskName": tn, "nodeName": nn, "realtime": d.realtime.mean(), "rank": 0},
        ignore_index=True)

for l, d in true_ranks_df.groupby(by=["wfName", "taskName"]):
    wn, tn = l
    ranks = rankdata(d.realtime.values, method="min")
    d["rank"] = ranks
    true_ranks_df.update(d)

In [ ]:
with db_actions.connect() as conn:
    scores_df = pds.read_sql('SELECT * FROM "nodeBenchmarkTransposedScores"', conn)


In [ ]:
scores_df["nodeName"] = scores_df["nodeConfig"].transform(lambda x: nodeIDLUT[x])
scores_df["nodeGroup"] = scores_df["nodeName"].transform(lambda x: awsInstanceGroupLUT[x.split(".")[0]])

In [ ]:
groupOrder = ["general", "compute", "memory", "storage"]
nodeOrder = list(pds.unique(dF.nodeName))

In [ ]:
benchmarkCols = [
    "build-linux-kernel1",
    "fio2",
    "fio3",
    "fio4",
    "fio5",
    "fio6",
    "fio7",
    "fio8",
    "fio9",
    "iperf10",
    "iperf11",
    "iperf12",
    "iperf13",
    "john-the-ripper14",
    "john-the-ripper15",
    "ramspeed16",
    "ramspeed17",
    "ramspeed18",
    "ramspeed19",
    "ramspeed20",
    "ramspeed21",
    "ramspeed22",
    "ramspeed23",
    "ramspeed24",
    "ramspeed25",
    "stream26",
    "stream27",
    "stream28",
    "stream29"]

In [ ]:
def find_group_protos(benchmarkCols=benchmarkCols, scores_dF=scores_df, groupOrder=groupOrder):
    scores = dict()
    nodeGroups = [scores_dF.query("nodeGroup==@gn").nodeName.values for gn in groupOrder]
    nodeGroupBenchmarkAverages = [np.average([scores_dF.query("nodeName==@nn")[benchmarkCols] for nn in ng], axis=0) for
                                  ng
                                  in nodeGroups]
    for c in itertools.product(*nodeGroups):
        proto_benchmarks = [scores_dF.query("nodeName == @pn")[benchmarkCols].values[0] for pn in c]
        distance_between_protos = 0
        for i, pn1 in enumerate(proto_benchmarks[:-1]):
            for pn2 in proto_benchmarks[i + 1:]:
                distance_between_protos += np.linalg.norm(pn1 - pn2)
        distances_from_group_averages = [np.linalg.norm(bs - ngba) for bs, ngba in
                                         zip(proto_benchmarks, nodeGroupBenchmarkAverages)]
        scores[c] = (distance_between_protos, np.sum(distances_from_group_averages))
    factor = sum(x[0] for x in scores.values()) / sum(x[1] for x in scores.values())
    scores = {
        k: v[0] + factor * v[1] for k, v in scores.items()
    }
    cs, ds = scores.keys(), scores.values()
    return [x[0] for x in sorted(zip(cs, ds), key=lambda x: x[1])]

In [ ]:
bestGroupPrototypes = (
    'm5.xlarge', 'c5.xlarge', 'r5.xlarge', 'i3.2xlarge')  # = sorted(zip(cs, ds), key=lambda x: x[1])[0][0]
bestGroupPrototypesLUT = {
    awsInstanceGroupLUT[k.split(".")[0]]: k for k in bestGroupPrototypes
}

In [ ]:
print(bestGroupPrototypesLUT)

# Produce Group Rankings

In [40]:
def makeGroupRankings(prototypes, instanceGroupLUT=awsInstanceGroupLUT, base_dF=dF, groupOrder=groupOrder):
    try:
        prototypesLUT = {
            instanceGroupLUT[k.split(".")[0]]: k for k in prototypes
        }
    except:
        prototypesLUT = {
            instanceGroupLUT[k]: k for k in prototypes
        }
    groupPerformances = dict()
    for g in base_dF.groupby(by=["nodeGroup", "wfName", "taskName"]):
        l, d = g
        ng, wfn, tn = l
        if (wfn, tn) not in groupPerformances.keys():
            groupPerformances[(wfn, tn)] = dict()
        groupProto = prototypesLUT[ng]
        groupPerformances[(wfn, tn)][ng] = d.query("nodeName == @groupProto").realtime.mean()
        #groupPerformances[l] = d.realtime.mean()
    groupRankings = {"wfName": [], "taskName": [], **{o: [] for o in groupOrder}}
    for k, v in groupPerformances.items():
        wfn, tn = k
        performances = [v[i] for i in groupOrder]
        ranks = rankdata(performances, method="min")
        groupRankings["wfName"].append(wfn)
        groupRankings["taskName"].append(tn)
        for o, r in zip(groupOrder, ranks):
            groupRankings[o].append(r)
    return pds.DataFrame(groupRankings)


#### Produce intragroup node rankings

In [35]:
def makeNodeRankingsInGroups(groupRankings, base_dF=dF, groupOrder=groupOrder, nodesPerGroup=nodesPerGroup):
    tmp = pds.DataFrame(
        columns=["group", "nodeName", "wfName", "taskName", "realtime", "groupRank", "intraGroupRank", "globalRank",
                 "trueRank"])
    tmp2 = {"group": [], "nodeName": [], "wfName": [], "taskName": [], "realtime": []}
    for g in base_dF.groupby(by=["nodeGroup", "wfName", "taskName", "nodeName"]):
        l, d = g
        ng, wfn, tn, nn = l
        tmp2["group"].append(ng)
        tmp2["nodeName"].append(nn)
        tmp2["wfName"].append(wfn)
        tmp2["taskName"].append(tn)
        tmp2["realtime"].append(d.realtime.mean())
    tmp = tmp.append(pds.DataFrame(tmp2), ignore_index=True)
    del tmp2

    for l, d in tmp.groupby(["group", "wfName", "taskName"]):
        gr, wfn, tn = l
        ra = list(rankdata(d.realtime, method="min"))
        cache = groupRankings.query("wfName==@wfn and taskName==@tn")
        ar = 0
        grr = cache[gr].values[0]
        for k, v in sorted(list(zip(groupOrder, (cache[k].values[0] for k in groupOrder))), key=lambda x: x[1]):
            if v >= grr:
                break
            ar += nodesPerGroup[k]
        # d["intraGroupRank"] = ra
        # d["groupRank"] = [grr] * len(ra)
        # d["globalRank"] = np.array([ar] * len(ra)) + np.array(ra)
        tmp.update(pds.DataFrame({"intraGroupRank": ra, "groupRank": [grr] * len(ra),
                                  "globalRank": np.array([ar] * len(ra)) + np.array(ra)}, index=d.index))

    for l, d in tmp.groupby(["wfName", "taskName"]):
        tr = list(rankdata(d.realtime, method="min"))
        #d["trueRank"] = tr
        tmp.update(pds.DataFrame({"trueRank": tr}, index=d.index))

    return tmp

In [ ]:
tmp3 = makeNodeRankingsInGroups(makeGroupRankings(bestGroupPrototypes))

In [ ]:
export = pds.DataFrame(
    {"wfName": tmp3.wfName, "taskName": tmp3.taskName, "nodeName": tmp3.nodeName, "rank": tmp3.globalRank})

In [ ]:
export.to_csv("no_ML_ranks.csv", index=False)

In [ ]:
def scoreRankings(ranking_df):
    scores = []
    for l, d in ranking_df.groupby(by=["wfName", "taskName"]):
        scores.append(commons.rankingScore(d.globalRank.values, d.trueRank.values))
    return np.average(scores), np.std(scores)

In [ ]:
print(scoreRankings(tmp3))

# Bruteforce test all prototype selections

In [ ]:
def do_one(x):
    return x, scoreRankings(makeNodeRankingsInGroups(makeGroupRankings(x)))


nodeGroups = [scores_dF.query("nodeGroup==@gn").nodeName.values for gn in groupOrder]

with Pool() as pool:
    results = pool.map(do_one, itertools.product(*nodeGroups))
    prototypeSelectionScores = {c: r for c, r in results}

In [ ]:
print(sorted(zip(prototypeSelectionScores.keys(), prototypeSelectionScores.values()),
             key=lambda x: prototypeSelectionScores[x[0]][0], reverse=True)[:3])

# New Instance Type Clusterings

In [44]:
def make_clustered_ranking(instanceGroupLUT=awsInstanceGroupLUT):
    clustered_dF = dF.copy()
    clustered_dF["nodeGroup"] = clustered_dF["nodeName"].transform(lambda x: instanceGroupLUT[x])
    clustered_groupOrder = pds.unique(clustered_dF.nodeGroup)
    clustered_scores_df = scores_df.copy()
    clustered_scores_df["nodeGroup"] = clustered_scores_df["nodeName"].transform(lambda x: instanceGroupLUT[x])
    bestClusteredGroupPrototypes = find_group_protos(scores_dF=clustered_scores_df, groupOrder=clustered_groupOrder)[0]
    return makeNodeRankingsInGroups(
        makeGroupRankings(prototypes=bestClusteredGroupPrototypes, instanceGroupLUT=instanceGroupLUT,
                          base_dF=clustered_dF, groupOrder=clustered_groupOrder),
        base_dF=clustered_dF,
        groupOrder=clustered_groupOrder,
        nodesPerGroup={l: len(d.unique()) for l, d in clustered_dF.groupby("nodeGroup").nodeName})

In [45]:
def score_clustering(instanceGroupLUT=awsInstanceGroupLUT):
    return scoreRankings(make_clustered_ranking(instanceGroupLUT=instanceGroupLUT))

In [46]:
instanceGroupLUT = {nn: c for nn, c in zip(scores_df.nodeName.values,
                                           AffinityPropagation(random_state=None).fit_predict(
                                               scores_df[benchmarkCols]))}
print(score_clustering(instanceGroupLUT))

(0.7407233769165323, 0.06563691837236124)


In [43]:
for n_clusters in range(2, 11):
    print(n_clusters, score_clustering({nn: c for nn, c in zip(scores_df.nodeName.values,
                                                               KMeans(n_clusters).fit_predict(
                                                                   scores_df[benchmarkCols]))}
                                       ))

2 (0.7702939695438523, 0.043496625716111896)
3 (0.8059071729957805, 0.04908270000854898)
4 (0.8293656995016583, 0.04422357750589898)
5 (0.7662308346790295, 0.05671507707957268)
6 (0.7812158149710893, 0.05999482445909217)
7 (0.7688006806619091, 0.0662614861873288)
8 (0.7654668264138493, 0.06002503000106251)
9 (0.8025559549235124, 0.05052909242978613)
10 (0.8025038634508864, 0.05019454221858715)


In [49]:
tmp3 = make_clustered_ranking({nn: c for nn, c in zip(scores_df.nodeName.values,
                                                               KMeans(n_clusters=4).fit_predict(
                                                                   scores_df[benchmarkCols]))}
                                       )
print(scoreRankings(tmp3))

(0.8293656995016583, 0.04422357750589898)


In [48]:
export = pds.DataFrame(
    {"wfName": tmp3.wfName, "taskName": tmp3.taskName, "nodeName": tmp3.nodeName, "rank": tmp3.globalRank})
export.to_csv("no_ML_ranks.csv", index=False)